In [1]:
import numpy as np
import dezero
import dezero.functions as F
import dezero.layers as L
import dezero.models as M
from dezero.models import Model

In [2]:
from dezero import cuda
from typing import List, Optional, Tuple, Any
import math

class StrokesDataset(dezero.DataLoader):
    def __init__(self, data, batch_size, max_seq_length: int, scale: Optional[float] = None, shuffle=True, gpu=False):
        stroke_data = []
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(data)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu
        
        xp = cuda.cupy if self.gpu else np
        
        for seq in data:
            # we will deem a sequence that is less than 10 as too short and thus ignore it
            if 10 < len(seq) <= max_seq_length:
                # clamp the delta x and delta y to [-1000, 1000]
                seq = np.minimum(seq, 1000)
                seq = np.maximum(seq, -1000)
                
                seq = np.array(seq, dtype=np.float32)
                stroke_data.append(seq)
        
        if scale is None:
            # calculate the scale factor
            # the scale factor is the standard deviation of the x and y coordinates
            # mean is not adjusted for simplicity
            # 0:2 means the first two columns of the array which are x and y coordinates
            scale = np.std(np.concatenate([np.ravel(s[:,0:2]) for s in stroke_data]))
        
        longest_seq_len = max([len(seq) for seq in stroke_data])
        
        # we add two extra columns to the dataset since we currently there are only 3 columns in the dataset
        # additional two columns are for changing the last point 1/0 to a one-hot vector
        temp_stroke_dataset = xp.zeros((len(stroke_data), longest_seq_len + 2, 5), dtype=np.float32)
        
        # self.mask is used to mark areas of the sequence that are not used
        # we first initialize it to zero
        temp_mask_dataset = xp.zeros((len(stroke_data), longest_seq_len + 1))
        
        self.dataset = []
        
        # start of sequence is [0, 0, 1, 0, 0]
        
        for i, seq in enumerate(stroke_data):
            seq = xp.array(seq, dtype=xp.float32)
            len_seq = len(seq)
            
            # we start from 1 to leave the first row for the start of sequence token
            temp_stroke_dataset[i, 1:len_seq + 1, 0:2] = seq[:, :2] / scale # this is the x and y coordinates
            temp_stroke_dataset[i, 1:len_seq + 1, 2] = 1 - seq[:, 2] # this is the pen down
            temp_stroke_dataset[i, 1:len_seq + 1, 3] = seq[:, 2] # this is the pen up
            temp_stroke_dataset[i, len_seq + 1, 4] = 1  # this is the end of sequence token
            temp_mask_dataset[i, :len_seq + 1] = 1 # mask is on until the end of the sequence 
            # self.mask is used to mark areas of the sequence that are not used
            # for example, if the sequence is shorter than the longest sequence, we use mask to ignore the rest of the sequence
            # an example of mask is [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        temp_stroke_dataset[:, 0, 2] = 1
        
        for i in range(len(stroke_data)):
            self.dataset.append([temp_stroke_dataset[i], temp_mask_dataset[i]])
        
        
        self.reset()

In [105]:
class BivariateGaussianMixture:
    def __init__(self, pi_logits, mu_x, mu_y, sigma_x, sigma_y, rho_xy):
        self.pi_logits = pi_logits
        self.mu_x = mu_x
        self.mu_y = mu_y
        self.sigma_x = sigma_x
        self.sigma_y = sigma_y
        self.rho_xy = rho_xy
    
    @property
    def n_distributions(self):
        return self.pi_logits.shape[-1]
    
    def set_temperature(self, temperature: float):
        self.pi_logits /= temperature
        self.sigma_x *= math.sqrt(temperature)
        self.sigma_y *= math.sqrt(temperature)
    
    def gaussian_pdf(self, x_delta, y_delta):
        # the result means the probability of y in the normal distribution
        # we check the probability of y in the normal distribution
        # if the probability is high, the result is close to 1
        
        while x_delta.shape != 

        norm1 = F.sub(x_delta, F.broadcast_to(self.mu_x, x_delta.shape))
        norm2 = F.sub(y_delta, F.broadcast_to(self.mu_y, y_delta.shape))
        xp = cuda.get_array_module(norm1)

        
        s1s2 = F.mul(self.sigma_x, self.sigma_y)
        
        # This is from: https://github.com/hardmaru/write-rnn-tensorflow/blob/master/model.py
        # z = tf.square(tf.div(norm1, s1)) + tf.square(tf.div(norm2, s2))
        #     - 2 * tf.div(tf.multiply(rho, tf.multiply(norm1, norm2)), s1s2)
         
        # below is the deconstruction of the above linez
        z_first_term = F.pow(F.div(norm1, self.sigma_x), 2)
        z_second_term = F.pow(F.div(norm2, self.sigma_y), 2)
        z_last_term_inner = F.mul(self.rho_xy, F.mul(norm1, norm2))
        z_last_term_middle = F.div(z_last_term_inner, s1s2)
        tmp_z = np.ones(z_last_term_middle.shape) * -2
        z_last_term = F.mul(tmp_z, z_last_term_middle)
        z = F.add(F.add(z_first_term, z_second_term), z_last_term)
        negRho = F.sub(np.ones(self.rho_xy.shape), F.pow(self.rho_xy, 2))

        
        result = F.exp(F.div(-z, 2 * negRho))
        deno_first_term = np.ones(self.sigma_x.shape) * 2 * math.pi
        denom_second_term = F.mul(s1s2, F.pow(negRho, 0.5))
        denom = F.mul(deno_first_term, denom_second_term)
        result = F.div(result, denom)
        
        return result

    # x1_data and x2_data are the real x and y coordinates of the stroke
    def get_lossfunc(self, x_delta, y_delta):
        result0 = self.gaussian_pdf(x_delta, y_delta)
        
        elipson = 1e-20
        # check if result0 has inf or nan
        result1 = F.mul(result0, self.pi_logits)
        result1 = F.sum(result1, axis=1, keepdims=True)
        result1 = -F.log(result1)
        
        return F.mean(result1)
    
    def get_pi_idx(self, x):
        pdf = self.pi_logits
        N = self.size
        accumulate = 0
        for i in range(0, N):
            accumulate += pdf[i]
            if accumulate >= x:
                return i
        print('error with sampling ensemble')
        return -1
    
    # M means the number of samples
    def sample(self, count, M=15):
        xp = cuda.get_array_module(self.pi_logits)
        # get the index of the distribution
        
        result = xp.random.rand(count, M, 2) # initially random [0,1]
        # we will get result for delta_x and delta_y
        rn = xp.random.rand(count, M, 2) 
        mu = 0
        std = 0
        idx = 0
        
        for j in range(M):
            for i in range(count):
                idx = self.get_pi_idx(result[i, j, 0])
                mu = [self.mu_x[i, idx], self.mu_y[i, idx]]
                std = [self.sigma_x[i, idx], self.sigma_y[i, idx]]
                rho = self.rho_xy[i, idx]
                
                result[i, j, 0:2] = mu + rn[i, j] * std
        return result

In [88]:
data = np.load('./data/sketchrnn_apple.npz', encoding='latin1', allow_pickle=True)

strokes = StrokesDataset(data['train'], batch_size=4, max_seq_length=200, gpu=False, shuffle=False)
print(strokes.data_size)
# first item

x, t = strokes.__next__()
print(x.shape, t.shape) # x is the stroke, t is the mask (x has one more column than t)
print(x[0], t[0])

# check if the mask is working
batch_size = x.shape[0]

for i in range(batch_size):
    mask_zero_id = np.where(t[i] == 0)[0]
    # first id
    first_id = mask_zero_id[0]
    stroke_end_id = np.where(x[i, :, 4] == 1)[0]
    first_stroke_end_id = stroke_end_id[0]
    
    print(first_id, first_stroke_end_id)

70000
(4, 64, 5) (4, 63)
[[ 0.          0.          1.          0.          0.        ]
 [-0.5628141  -0.5959208   1.          0.          0.        ]
 [-0.62902755 -0.46349397  1.          0.          0.        ]
 [-1.0594149  -0.06621343  1.          0.          0.        ]
 [-0.43038726  0.16553356  1.          0.          0.        ]
 [-0.5628141   0.36417383  1.          0.          0.        ]
 [-0.2979604   0.36417383  1.          0.          0.        ]
 [-0.2979604   1.5891222   1.          0.          0.        ]
 [ 0.09932014  1.0594149   1.          0.          0.        ]
 [ 0.2979604   0.4966007   1.          0.          0.        ]
 [ 2.1188297   2.3174698   1.          0.          0.        ]
 [ 0.2979604   1.4898021   1.          0.          0.        ]
 [ 0.33106712  0.33106712  1.          0.          0.        ]
 [ 0.62902755  0.09932014  1.          0.          0.        ]
 [ 0.39728054 -0.4966007   1.          0.          0.        ]
 [ 0.72834766  0.          1. 

In [89]:
class Encoder(Model):
    def __init__(self, d_z, hidden_size):
        super().__init__()
        self.lstm = L.LSTM(in_size=5, hidden_size=hidden_size)
        self.mu_head = L.Linear(in_size=hidden_size, out_size=d_z)
        self.sigma_head = L.Linear(in_size=hidden_size, out_size=d_z)

        self.hidden_size = hidden_size
        self.d_z = d_z

    def forward(self, inputs):
        hidden = self.lstm(inputs)

        mu = self.mu_head(hidden)
        sigma_hat = self.sigma_head(hidden)
        sigma = F.exp(sigma_hat / 2.)

        xp = dezero.cuda.get_array_module(mu.data)
        z = mu + sigma * xp.random.normal(0, 1, mu.shape)
        return z, mu, sigma

In [90]:
class Decoder(Model):
    def __init__(self, d_z, hidden_size, n_distributions):
        super().__init__()
        self.lstm = L.LSTM(in_size=d_z+5, hidden_size=hidden_size)

        self.init_state = L.Linear(in_size=d_z, out_size=2*hidden_size)

        self.mixtures = L.Linear(in_size=hidden_size, out_size=6 * n_distributions)
        self.q_head = L.Linear(in_size=hidden_size, out_size=3)

        self.n_distributions = n_distributions
        self.hidden_size = hidden_size

    def forward(self, inputs, z, state=None):
        xp = dezero.cuda.get_array_module(x.data)
        if state == None:
            h, c = xp.split(self.init_state(z).data, 2, axis=2)
            state = (h, c)

        self.lstm.set_state(state)

        # hidden Needs to chagned to output of lstm
        hidden = self.lstm(inputs)

        q_logits = F.log_softmax(self.q_head(hidden))

        pi_logits, mu_x, mu_y, sigma_x, sigma_y, rho_xy = xp.split(self.mixtures(hidden).data, 6, 2)

        bgm = BivariateGaussianMixture(pi_logits, mu_x, mu_y, F.exp(sigma_x), F.exp(sigma_y), F.tanh(rho_xy))
        return bgm, q_logits
        

In [112]:
def ReconstructionLoss(target, mask, bgm, q_logits):
    xp = cuda.get_array_module(mask)
    xy = target[:, :, 0:2]
    xy = xy[:, :, xp.newaxis, :]
    xy = xp.tile(xy, (1, 1, bgm.n_distributions, 1))
    
    # expanded_shape = (xy.shape[0], xy.shape[1], bgm.n_distributions, xy.shape[3])
    
    x = xy[:,:,:,0]
    y = xy[:,:,:,0]

    print(xy.shape, target.shape, x.shape, y.shape)
    
    loss_stroke = F.mul(bgm.get_lossfunc(x, y), mask)
    
    loss_pen = -F.mean(F.mul(target[:,:,2:], q_logits))
    
    return F.add(loss_stroke, loss_pen)

def KLDivergenceLoss(mu, sigma):
    xp = cuda.get_array_module(mu)
    tmp = xp.ones(sigma.shape)
    inner_1 = F.add(tmp, sigma)
    inner_2 = F.add(F.pow(mu, 2), F.exp(sigma))
    inner = F.sub(inner_1, inner_2)
    tmp2 = xp.ones(inner.shape) * -2
    return F.mean(F.div(inner, tmp2))
        

In [92]:
class VAE(Model):
    def __init__(self, d_z, enc_hidden_size, dec_hidden_size, n_distributions):
        super().__init__()
        self.encoder = Encoder(d_z, enc_hidden_size)
        self.decoder = Decoder(d_z, dec_hidden_size, n_distributions)

    def forward(self, x, t):
        z, mu, sigma = self.encoder(x)

        inputs = np.concatenate((x.data, z.data), axis=2)
        inputs = dezero.as_variable(inputs)
        bgm, q_logits = self.decoder(inputs, z)

        kl_loss = KLDivergenceLoss(mu, sigma)
        recon_loss = ReconstructionLoss(x, t, bgm, q_logits)
        
        return kl_loss + recon_loss

In [93]:
d_z = 4
enc_hidden_size = 32
dec_hidden_size = 64
n_distributions = 8
epochs = 10

In [94]:
encoder = Encoder(d_z, enc_hidden_size)
z, mu, sigma = encoder(x)
z.shape ,mu.shape, sigma.shape

((4, 64, 4), (4, 64, 4), (4, 64, 4))

In [100]:
decoder = Decoder(d_z, dec_hidden_size, n_distributions)

print(x.shape, z.shape)

# x의 마지막 요소를 제외하고 가져와서 inputs 생성
inputs = np.concatenate((x.data, z.data), axis=2)
inputs = dezero.as_variable(inputs)

print(inputs.shape)
decoder(inputs, z)
pass


(4, 64, 5) (4, 64, 4)
(4, 64, 9)


In [113]:
model = VAE(d_z, enc_hidden_size, dec_hidden_size, n_distributions)

for epoch in range(epochs):
    for i in range(strokes.max_iter):
        x, t = strokes.__next__()
        loss = model(x, t)
        model.cleargrads()
        loss.backward()
        model.update()
        print(loss)
        
    print(f"Epoch {epoch} finished")

(4, 64, 8, 2) (4, 64, 5) (4, 64, 8) (4, 64, 8)


c:\Users\user\Desktop\흠\Sketch-RNN\dezero\functions.py:115: RuntimeWarning: invalid value encountered in log
  y = xp.log(x)


ValueError: shapes (64,64,4) and (4,64,3) not aligned: 4 (dim 2) != 64 (dim 1)